In [1]:
from models import VariationalAutoencoder, Autoencoder, ConvAutoencoder, ConvVAE, MyModel
from data_utils import CustomImageDataset
import torch
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

Using device: cuda


In [3]:
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.CenterCrop(28),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0), std=(1)),
     transforms.GaussianBlur(kernel_size=(5, 5)),
     ])

In [4]:
proliv_data_path = "./homework_2_dataset/dataset/proliv"
proliv_dataset = CustomImageDataset(os.path.join(proliv_data_path, "annotation.txt"), os.path.join(proliv_data_path, "imgs"), transform=transform)
proliv_dataloader = DataLoader(proliv_dataset, batch_size=1, shuffle=True)

## Определения порога loss

В качестве порога Loss будем брать значение с наименьшим True Positive Rate > 0.91, чтобы максимизировать True Negative Rate

### AE

In [5]:
ae = Autoencoder(16).to(device) # GPU
ae.load_state_dict(torch.load("./autoencoders/AE"))
ae.eval()

Autoencoder(
  (encoder): Encoder(
    (linear1): Linear(in_features=2352, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=16, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=2352, bias=True)
  )
)

In [6]:
losses = []
true = []
for x, y in proliv_dataloader:
    true.append(int(y[0]))
    x = x.to(device) # GPU
    reconstruction = ae(x)
    loss = ((x - reconstruction)**2).sum()
    losses.append(loss.item())


thresolds, tp_rate = [], []
best_tresholds = []
for thresh in range(10, 80, 1):
    tp, fp, tn, fn = 0, 0, 0, 0
    for loss, y in zip(losses, true):
        if loss >= (thresh / 10):
            y_pred = 1
        else:
            y_pred = 0
        
        if y == 1 and y_pred == 1:
            tp += 1
        elif y == 1 and y_pred == 0:
            fn += 1
        elif y == 0 and y_pred == 1:
            fp += 1
        elif y == 0 and y_pred == 0:
            tn += 1
    
    thresolds.append(thresh / 10)
    tp_rate.append(tp / (tp + fn))

    if tp / (tp + fn) > 0.91:
        best_tresholds.append(thresh / 10)
        # print(f"threshold = {(thresh / 10):.2f}: True Positive Rate: {(tp / (tp + fn)):.2f}")

if len(best_tresholds) > 0:
    print(f"choosen threshold: {best_tresholds[-1]}")
else:
    print("no possible thresholds")

choosen threshold: 4.5


### VAE

In [7]:
vae = VariationalAutoencoder(16).to(device) # GPU
vae.load_state_dict(torch.load("./autoencoders/VAE"))
vae.eval()

VariationalAutoencoder(
  (encoder): VariationalEncoder(
    (linear1): Linear(in_features=2352, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=16, bias=True)
    (linear3): Linear(in_features=512, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=16, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=2352, bias=True)
  )
)

In [8]:
losses = []
true = []
for x, y in proliv_dataloader:
    true.append(int(y[0]))
    x = x.to(device) # GPU
    reconstruction = vae(x)
    loss = ((x - reconstruction)**2).sum() + vae.encoder.kl
    losses.append(loss.item())


thresolds, tp_rate = [], []
best_tresholds = []
for thresh in range(10, 2000, 1):
    tp, fp, tn, fn = 0, 0, 0, 0
    for loss, y in zip(losses, true):
        if loss >= (thresh / 10):
            y_pred = 1
        else:
            y_pred = 0
        
        if y == 1 and y_pred == 1:
            tp += 1
        elif y == 1 and y_pred == 0:
            fn += 1
        elif y == 0 and y_pred == 1:
            fp += 1
        elif y == 0 and y_pred == 0:
            tn += 1
    
    thresolds.append(thresh / 10)
    tp_rate.append(tp / (tp + fn))

    if tp / (tp + fn) > 0.91:
        best_tresholds.append(thresh / 10)
        # print(f"threshold = {(thresh / 10):.2f}: True Positive Rate: {(tp / (tp + fn)):.2f}")

if len(best_tresholds) > 0:
    print(f"choosen threshold: {best_tresholds[-1]}")
else:
    print("no possible thresholds")

choosen threshold: 17.3


### convolutional AE

In [9]:
conv = ConvAutoencoder().to(device) # GPU
conv.load_state_dict(torch.load("./autoencoders/conv_ae"))
conv.eval()

ConvAutoencoder(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(4, 16, kernel_size=(2, 2), stride=(2, 2))
  (t_conv2): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2))
)

In [10]:
losses = []
true = []
for x, y in proliv_dataloader:
    true.append(int(y[0]))
    x = x.to(device) # GPU
    reconstruction = conv(x)
    loss = ((x - reconstruction)**2).sum()
    losses.append(loss.item())


thresolds, tp_rate = [], []
best_tresholds = []
for thresh in range(10, 200, 1):
    tp, fp, tn, fn = 0, 0, 0, 0
    for loss, y in zip(losses, true):
        if loss >= (thresh / 10):
            y_pred = 1
        else:
            y_pred = 0
        
        if y == 1 and y_pred == 1:
            tp += 1
        elif y == 1 and y_pred == 0:
            fn += 1
        elif y == 0 and y_pred == 1:
            fp += 1
        elif y == 0 and y_pred == 0:
            tn += 1
    
    thresolds.append(thresh / 10)
    tp_rate.append(tp / (tp + fn))

    if tp / (tp + fn) > 0.91:
        best_tresholds.append(thresh / 10)
        # print(f"threshold = {(thresh / 10):.2f}: True Positive Rate: {(tp / (tp + fn)):.2f}")

if len(best_tresholds) > 0:
    print(f"choosen threshold: {best_tresholds[-1]}")
else:
    print("no possible thresholds")

choosen threshold: 2.5


### convolutional VAE

In [11]:
conv_vae = ConvVAE().to(device) # GPU
conv_vae.load_state_dict(torch.load("./autoencoders/conv_vae"))
conv_vae.eval()

ConvVAE(
  (criterion): MSELoss()
  (Encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.25, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.25, inplace=False)
    (12): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.25, inplace=False)
  )
  (fc1): Linear(in_features=256, out_features=32, bias=True)
  

In [12]:
losses = []
true = []
for x, y in proliv_dataloader:
    true.append(int(y[0]))
    x = x.to(device) # GPU
    reconstruction, mu, logvar = conv_vae(x)
    loss = conv_vae.loss(reconstruction, mu, logvar, x)
    losses.append(loss.item())


thresolds, tp_rate = [], []
best_tresholds = []
for thresh in range(0, 150, 1):
    tp, fp, tn, fn = 0, 0, 0, 0
    for loss, y in zip(losses, true):
        if loss >= (thresh / 1000):
            y_pred = 1
        else:
            y_pred = 0
        
        if y == 1 and y_pred == 1:
            tp += 1
        elif y == 1 and y_pred == 0:
            fn += 1
        elif y == 0 and y_pred == 1:
            fp += 1
        elif y == 0 and y_pred == 0:
            tn += 1
    
    thresolds.append(thresh / 1000)
    tp_rate.append(tp / (tp + fn))

    if tp / (tp + fn) > 0.91:
        best_tresholds.append(thresh / 1000)
        # print(f"threshold = {(thresh / 1000):.3f}: True Positive Rate: {(tp / (tp + fn)):.2f}")

if len(best_tresholds) > 0:
    print(f"choosen threshold: {best_tresholds[-1]}")
else:
    print("no possible thresholds")

choosen threshold: 0.019


### MyModel

In [13]:
mm = MyModel().to(device) # GPU
mm.load_state_dict(torch.load("./autoencoders/my_model"))
mm.eval()

MyModel(
  (criterion): MSELoss()
  (Encoder): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (fc1): Linear(in_features=1176, out_features=512, bias=True)
  (fc_mu): Linear(in_features=512, out_features=32, bias=True)
  (fc_log_var): Linear(in_features=512, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1176, bias=True)
  (Decoder): Sequential(
    (0): Linear(in_features=1176, out_features=2352, bias=True)
    (1): Sigmoid()
  )
)

In [14]:
losses = []
true = []
for x, y in proliv_dataloader:
    true.append(int(y[0]))
    x = x.to(device) # GPU
    reconstruction, mu, logvar = mm(x)
    loss = mm.loss(reconstruction, x)
    losses.append(loss.item())


thresolds, tp_rate = [], []
best_tresholds = []
for thresh in range(0, 100, 1):
    tp, fp, tn, fn = 0, 0, 0, 0
    for loss, y in zip(losses, true):
        if loss >= (thresh / 10000):
            y_pred = 1
        else:
            y_pred = 0
        
        if y == 1 and y_pred == 1:
            tp += 1
        elif y == 1 and y_pred == 0:
            fn += 1
        elif y == 0 and y_pred == 1:
            fp += 1
        elif y == 0 and y_pred == 0:
            tn += 1
    
    thresolds.append(thresh / 10000)
    tp_rate.append(tp / (tp + fn))

    if tp / (tp + fn) > 0.91:
        best_tresholds.append(thresh / 10000)
        # print(f"threshold = {(thresh / 10000):.4f}: True Positive Rate: {(tp / (tp + fn)):.2f}")

if len(best_tresholds) > 0:
    print(f"choosen threshold: {best_tresholds[-1]}")
else:
    print("no possible thresholds")

choosen threshold: 0.002
